In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import yfinance as yf
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)



import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

import sys
sys.path.append('/Users/educontreras/PycharmProjects/Quantitative-Finance/AlgoTrading_E_Chang')
import alpha_vantage
from Indicators.Momentum.Multi_Spectral_RSI_Divergence import MS_RSI_DIVERGENCE

In [2]:
M = MS_RSI_DIVERGENCE("AAPL", "2005-01-01")
M.compute_divergence(5, 10,80,5)
M.compute_profits(entry_level=5, trend_confirmation_days=2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## When are we losing money? Long-only version

Here we aiming to understand what happens when we enter a long position, when are we losing money and why?

In [ ]:
M.

In [3]:
mask = M.long_positions == 1
longs = M.long_positions[mask].dropna()
longs

,Long
Date,
2005-03-15,1.0
2005-03-16,1.0
2005-03-17,1.0
2005-03-18,1.0
2005-03-21,1.0
...,...
2024-03-12,1.0
2024-04-25,1.0
2024-04-26,1.0


In [ ]:
fig = go.Figure()
aux_df = pd.merge(M.prices, M.prices[M.prices.index.isin(longs.index)], left_index = True, right_index = True, how = "outer")
aux_df.columns = ["Total Price", "Price when Long"]
fig.add_trace(
    go.Scatter(
        x = aux_df.index,
        y = aux_df.iloc[:, 0].values.flatten(),
        mode = "lines",
        name = "Long Equity Curve"
        
    )
)
fig.add_trace(
    go.Scatter(
        x = aux_df.index,
        y = aux_df.iloc[:, 1].values.flatten(),
        name = "Price when Long",
        line = dict(
            color = "darkgreen", width = 3
        )
    )
)
fig.show()

In [ ]:
M.long_account_curve.iplot()

In [8]:
short_days = [element for element in M.short_positions[M.short_positions.diff() != 0]]


In [38]:
df = pd.merge(M.indicator["Open"].loc[:, "Lvl"], M.indicator["Close"].loc[:, "Lvl"], left_index = True, right_index = True)
df.iplot()

In [4]:
#Cuando haces diff: 1 es la entrada en long y -1 es la entrada en short

longs_entry_exit = M.long_positions.diff()
entry = longs_entry_exit[longs_entry_exit == 1].dropna().index.tolist()
exit = longs_entry_exit[longs_entry_exit == -1].dropna().index.tolist()
trades = list(zip(entry,exit))
success = 0
for en, ex in trades:
    profit_entry = M.long_account_curve.loc[en]
    profit_exit = M.long_account_curve.loc[ex]
    if profit_exit>profit_entry:
        success += 1
print(success/len(trades))

0.5454545454545454


In [10]:
list(set(M.short_positions.values.flatten().tolist()))


[0.0, -1.0]

In [27]:
M.long_account_curve.loc[entry[1]]

-4.638543950888585

In [23]:
M.long_account_curve

Date
2005-01-12      0.000000
2005-01-13      0.000000
2005-01-14      0.000000
2005-01-18      0.000000
2005-01-19      0.000000
                 ...    
2024-04-19    580.452133
2024-04-22    580.452133
2024-04-23    580.452133
2024-04-24    580.452133
2024-04-25    605.688213
Length: 4854, dtype: float64

In [21]:
M.indicator["Open"]

,Lvl,Trend,Trend Change,# Days in Trend
Date,,,,
2005-01-11,-0.343852,1,NaN,0
2005-01-12,-1.040204,-1,-1.0,1
2005-01-13,-1.631010,-1,0.0,0
2005-01-14,2.084599,1,1.0,2
2005-01-18,3.824582,1,0.0,0
...,...,...,...,...
2024-04-19,-8.807064,-1,0.0,0
2024-04-22,-12.189904,-1,0.0,0
2024-04-23,-9.388912,1,1.0,1
